In [1]:
from py2neo import Graph, Path
from py2neo import Node, Relationship
from pymongo import MongoClient
import json
import csv

config = json.load(open("../graph/config.json"))

graph = Graph("http://"+config["user"]+":"+config["password"]+"@"+config["server"]+":7474")

client = MongoClient()
client = MongoClient(config["server"], 27017)
db = client.Biomedical

In [2]:
from os import listdir
#List the csv files as its transported from another server as csv
#This can use the hdfs system if the crawling data is saved to hdfs
files = listdir('./')

In [3]:
def save_to_mongo(row):
    person = row[0]
    school = row[1]
    publication = []
    coworker = []
    #Add person including name, university, publications and coworkers
    result = {}
    result['category'] = 'Person'
    for j in range(10):
        if len(row[j*3+2]) != 0:
            publication.append(row[j*3+2])
            for i in range(2):
                if len(row[j*3+3+i]) != 0:
                    coworker.append(row[j*3+3+i])
    result['label'] = person
    result['publication'] = publication
    result['coworker'] = coworker
    result['institute'] = school
    #print('person',result)
    db.crawlper.insert_one(result)
    #Add publication node
    for j in range(len(publication)):
        result = {}
        result['category'] = 'Publication'
        author = [person]
        result['label'] = publication[j]
        for i in range(2):
            if len(row[j*3+3+i]) != 0:
                author.append(row[j*3+3+i])
        result['author'] = author
        #print('publication',result)
        db.crawlpub.insert_one(result)

In [9]:
def read_csv(file):
    result = []
    with open(file, newline='\n') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        flag = True
        for row in spamreader:
            #skip the first line
            if flag:
                flag = False
            else:
                result.append(row)
    return result

In [5]:
def mergedata():
    datafile = []
    for file in files:
        if file.split('.')[-1] == 'csv':
            datafile.append(file)
    for file in datafile:
        csvfile = read_csv('./'+file)
        for row in csvfile:
            save_to_mongo(row)

In [10]:
mergedata()

In [8]:
db.crawlper.remove({})
db.crawlpub.remove({})

F:\python\Lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.
F:\python\Lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  


{'n': 2336, 'ok': 1.0}

In [36]:
#This is the test of reading output
mergedata()

person {'category': 'Person', 'label': 'name', 'publication': ['publication0', 'publication1', 'publication2', 'publication3', 'publication4', 'publication5', 'publication6', 'publication7', 'publication8', 'publication9'], 'coworker': ['cowriter00', 'cowriter01', 'cowriter10', 'cowriter11', 'cowriter20', 'cowriter21', 'cowriter30', 'cowriter31', 'cowriter40', 'cowriter41', 'cowriter50', 'cowriter51', 'cowriter60', 'cowriter61', 'cowriter70', 'cowriter71', 'cowriter80', 'cowriter81', 'cowriter90', 'cowriter91'], 'institute': 'school'}
publication {'category': 'Publication', 'label': 'publication0', 'author': ['name', 'cowriter00', 'cowriter01']}
publication {'category': 'Publication', 'label': 'publication1', 'author': ['name', 'cowriter10', 'cowriter11']}
publication {'category': 'Publication', 'label': 'publication2', 'author': ['name', 'cowriter20', 'cowriter21']}
publication {'category': 'Publication', 'label': 'publication3', 'author': ['name', 'cowriter30', 'cowriter31']}
publica

publication {'category': 'Publication', 'label': 'Red blood cell aggregation and dissociation in shear flows simulated by lattice Boltzmann method', 'author': ['Popel A S', 'Zhang J', 'Johnson P C']}
publication {'category': 'Publication', 'label': 'Effects of erythrocyte deformability and aggregation on the cell free layer and apparent viscosity of microscopic blood flows', 'author': ['Popel A S', 'Zhang J', 'Johnson P C']}
publication {'category': 'Publication', 'label': 'Computational fluid dynamic simulation of aggregation of deformable cells in a shear flow', 'author': ['Popel A S', 'Bagchi P', 'Johnson P C']}
publication {'category': 'Publication', 'label': 'Contribution of red blood cell aggregation to venous vascular resistance in skeletal muscle', 'author': ['Popel A S', 'Cabel M', 'Meiselman H J']}
person {'category': 'Person', 'label': 'Shadmehr R', 'publication': ['Adaptive representation of dynamics during learning of a motor task', 'Neural correlates of motor memory conso

publication {'category': 'Publication', 'label': 'Antigen recognition by human T lymphocytes is linked to surface expression of the T3 molecular complex', 'author': ['Levine H', 'Reinherz E L', 'Meuer S']}
publication {'category': 'Publication', 'label': 'Diffuse interface model of diffusion-limited crystal growth', 'author': ['Levine H', 'Collins J B']}
publication {'category': 'Publication', 'label': 'Left ventricular compliance: mechanisms and clinical implications', 'author': ['Levine H', 'Gaasch W H', 'Quinones M A']}
publication {'category': 'Publication', 'label': 'Cooperative self-organization of microorganisms', 'author': ['Levine H', 'Ben-Jacob E', 'Cohen I']}
publication {'category': 'Publication', 'label': 'Self-organization in systems of self-propelled particles', 'author': ['Levine H', 'Rappel W J', 'Cohen I']}
publication {'category': 'Publication', 'label': 'Functional consequences of interleukin 2 receptor expression on resting human lymphocytes. Identification of a no

## Merge the data into the graph

In [5]:
def addpernodes():
    nodes = db.crawlper.find({})
    for node in nodes:
        query = 'MATCH (c) WHERE c.title=["' + node['label'] + '"] RETURN c'
        if len(graph.run(query).data()) == 0:
            gn = Node(node['category'],title = [node['label']])
            graph.create(gn)
        school = node['institute']
        query = 'MATCH (c) WHERE c.title=["' + school + '"] RETURN c'
        if len(graph.run(query).data()) == 0:
            gn = Node('Institute',title = [school])
            graph.create(gn)

                
def addpubnodes():
    nodes = db.crawlpub.find({})
    for node in nodes:
        try:
            query = 'MATCH (c) WHERE c.title=["' + node['label'] + '"] RETURN c'
            if len(graph.run(query).data()) == 0:
                gn = Node(node['category'],title = [node['label']])
                graph.create(gn)
            for per in node['author']:
                query = 'MATCH (c) WHERE c.title=["' + per + '"] RETURN c'
                if len(graph.run(query).data()) == 0:
                    gn = Node('Person',title = [node['label']])
                    graph.create(gn)
        except:
            print(node['label'],node['_id'])
    
    

#Merge these information into this knowledge graph
def addperedges():
    nodes = db.crawlper.find({})
    cats = ['Publication','Person']
    mapping={'Person':'coworker','Publication':'publication'}
    for node in nodes:
        #Add relation between peopel and institutes first
        try:
            cat = 'University'
            relate = 'Is work place'
            createR = 'MATCH (p:'+node['category']+' {title:["'+node['label']+'"]}),'+\
                    '(c:'+cat+' {title:["'+node['institute']+'"]})' +'CREATE (c)-[:`'+relate+'`]->(p)'
            graph.run(createR)
            for cat in cats:
                cate = mapping[cat]
                relate = 'Has '+cate
                for r in node[cate]:
                    createR = 'MATCH (p:'+node['category']+' {title:["'+node['label']+'"]}),'+\
                            '(c:'+cat+' {title:["'+r+'"]})' +'CREATE (c)-[:`'+relate+'`]->(p)'
                    graph.run(createR)
        except:
            print(node['label'],node['_id'])
                
                
def addpubedges():
    nodes = db.crawlpub.find({})
    for node in nodes:
        #Add relation between peopel and institutes first
        try:
            cat = 'Person'
            cate = 'author'
            relate = 'Has '+cate
            for r in node[cate]:
                createR = 'MATCH (p:'+node['category']+' {title:["'+node['label']+'"]}),'+\
                        '(c:'+cat+' {title:["'+r+'"]})' +'CREATE (c)-[:`'+relate+'`]->(p)'
                graph.run(createR)
        except:
            print(node['label'],node['_id'])

In [19]:
addpernodes()

In [5]:
addpubnodes()

"Intracoronary ultrasound in cardiac transplant recipients. In vivo evidence of"" angiographically silent"" intimal thickening" 5af3deb343bf572a202aaece
"Time Variability of the"" Quiet"" Sun Observed with TRACE. II. Physical Parameters_ Temperature Evolution_ and Energetics of EUV Nanoflares" 5af3deb943bf572a202aafc9


In [7]:
addperedges()

Danino T 5af3de9a43bf572a202aaac5
Popp R L 5af3deb343bf572a202aaec8
Nightingale R W 5af3deb943bf572a202aafc4


In [4]:
addpubedges()

"Intracoronary ultrasound in cardiac transplant recipients. In vivo evidence of"" angiographically silent"" intimal thickening" 5af3deb343bf572a202aaece
"Time Variability of the"" Quiet"" Sun Observed with TRACE. II. Physical Parameters_ Temperature Evolution_ and Energetics of EUV Nanoflares" 5af3deb943bf572a202aafc9
